In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

|Column|Type|日本語|
|:-----|:-----|:-----|
|DisclosureNumber|Int64|開示番号|
|DateCode|string|年月日_銘柄コード|
|Date|date|取引日※他のデータと結合するための日付|
|SecuritiesCode|Int64|銘柄コード|
|DisclosedDate|date|開示日|
|DisclosedTime|time|開示時間|
|DisclosedUnixTime|Int64|開示日時（Unix時間）|
|TypeOfDocument|string|開示文書種類|
|CurrentPeriodEndDate|date|四半期末|
|TypeOfCurrentPeriod|date|四半期※4Q、5Qは例外（決算期変更？）|
|CurrentFiscalYearStartDate|date|期初|
|CurrentFiscalYearEndDate|date|期末日|
|NetSales|Int64|売上高／1Qからの積み上げ※四半期のみではない|
|OperatingProfit|Int64|営業利益／1Qからの積み上げ※銀行や保険業はNaN|
|OrdinaryProfit|Int64|経常利益※会計基準によって、計算方法が違う|
|Profit|Int64|純利益※Consolidated＝連結|
|EarningsPerShare|float|一株当たり当期純利益|
|TotalAssets|Int64|総資産|
|Equity|Int64|株主資本|
|EquityToAssetRatio|float|株主資本比率|
|BookValuePerShare|float|一株当たり純資産|
|ResultDividendPerShare1stQuarter|float|第1四半期の1株あたり配当実績※ー：配当なし、0：0.1円以下|
|ResultDividendPerShare2ndQuarter|float|第2四半期の1株あたり配当実績※ー：配当なし、0：0.1円以下|
|ResultDividendPerShare3rdQuarter|float|第3四半期の1株あたり配当実績※ー：配当なし、0：0.1円以下|
|ResultDividendPerShareFiscalYearEnd|float|第4四半期の1株あたり配当実績※ー：配当なし、0：0.1円以下|
|ResultDividendPerShareAnnual|float|1株あたり年間配当実績※ー：配当なし、0：0.1円以下|
|ForecastDividendPerShare1stQuarter|float|第1四半期の1株あたり配当予想※ー：配当なし、0：0.1円以下|
|ForecastDividendPerShare2ndQuarter|float|第2四半期の1株あたり配当予想※ー：配当なし、0：0.1円以下|
|ForecastDividendPerShare3rdQuarter|float|第3四半期の1株あたり配当予想※ー：配当なし、0：0.1円以下|
|ForecastDividendPerShareFiscalYearEnd|float|第4四半期の1株あたり配当予想※ー：配当なし、0：0.1円以下|
|ForecastDividendPerShareAnnual|float|1株あたり年間配当予想※ー：配当なし、0：0.1円以下|
|ForecastNetSales|Int64|予想売上高|
|ForecastOperatingProfit|Int64|予想営業利益|
|ForecastOrdinaryProfit|Int64|予想経常利益|
|ForecastProfit|Int64|予想純利益|
|ForecastEarningsPerShare|float|予想一株当たり純利益|
|ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements|boolean|四半期財務諸表の作成に特有の会計処理の適用|
|MaterialChangesInSubsidiaries|boolean|期中における重要な子会社の異動|
|ChangesBasedOnRevisionsOfAccountingStandard|boolean|会計基準等の改正に伴う会計方針の変更|
|ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard|boolean|↑以外の会計方針の変更|
|ChangesInAccountingEstimates|boolean|会計上の見積りの変更 |
|RetrospectiveRestatement|boolean|修正再表示 |
|NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock|Int64|期末発行済株式数|
|NumberOfTreasuryStockAtTheEndOfFiscalYear|Int64|期末自己株式数|
|AverageNumberOfShares|Int64|期中平均株式数|


参考：日本取引所　決算短信作成要領・四半期決算短信作成要領
https://www.jpx.co.jp/equities/listed-co/format/summary/index.html

In [ ]:
financials = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv')

In [ ]:
# 開示文書種類について
typeOfDoc = financials["TypeOfDocument"]
typeOfDoc = typeOfDoc.drop_duplicates()
typeOfDoc

- 1Q, 2Q, 3Q: 四半期
- FY: 通期
- OtherPeriod: 四半期以外※4Qや5Qの数字。具体例↓
- IFRS, JP, US: 会計基準
- Consolidated, NonConsolidated: 連結、非連結
- REIT: 不動産投資信託※半期報告書が必要
- ForecastRevision: 業績予想の修正
- NumericalCorrection: 業績の訂正

In [ ]:
financials[financials["TypeOfCurrentPeriod"] == "5Q"]

3987エコモット　2020年8月期は17ヶ月
https://www.ecomott.co.jp/ir/news.html
https://ssl4.eir-parts.net/doc/3987/yuho_pdf/S100JHO8/00.pdf

# TODO
- 開示日から次の開示日までのstock_prices.csvデータと結合
- 各データのYonYやQonQ